In [1]:
#!pip freeze 
# !pip install -r requirements.txt
import numpy as np
import matplotlib.pyplot as plt 
#!pip install -U ipykernel
#[2]=Domain,steps,planes
#[3]=IntegScheme
#[4]=e plane matices 
#[5]=plot control disps
#[6]=Elasticity matrix vals
#[7]=Elastic stresses
#[8]=plot control stresses 
#[9]=sigma integrations for macro->mplane
#[10]=main function 

In [2]:
split=True
n_dim = 2 #number of dims in domain 
n_mp = 360 #number of microplanes in interpolation 
n_steps = 1000 #solver steps 
#ratio=step of defining actual domain
delta = np.identity(n_dim) #Kronecker δij
epsilon_11=np.linspace(0,0.01,n_steps)#define evenly the lin space from 0 to 0.01 with n_steps is the n in  nijk
#0.01 optional or otherwise raises DeprecationWarning when num cannot be safely interpreted as an integer
epsilon_ij=np.zeros((len(epsilon_11,),n_dim,n_dim))#creation of the finite 2d space
#question one-> since we use the ntr transormation and the space is also 2d there is no need for 3d?
#epsilon_ijk=np.zeros((len(epsilon_11,),n_dim,n_dim,n_dim))
epsilon_ij[:,0,0]=epsilon_11 #reshape,arrange,apprehend 
#epsilon_ij[:,0,0,0]=epsilon_11 for 3d
epsilon_ijcalc=epsilon_ij[0,:,:]#slicing for the calculations in the loops 


In [3]:
class IntegScheme():
    #n_mp=360
    alpha_list = np.linspace(0,2*np.pi,n_mp)#polar angle dom
    a_length=alpha_list.size 
    MPN = np.array([[np.cos(alpha),np.sin(alpha)] for alpha in alpha_list ])
    #alpha=float val?
    #weights of each microplane
    if n_dim==2:
        MPW = np.ones(n_mp) / n_mp * 2 
    elif n_dim==3:
        MPW = np.ones(n_mp) / n_mp * 6
#     else: 
    MPM = np.zeros_like(MPN)
    delta= np.identity(n_dim) #should be already set?
    MPNN_nij = np.einsum('ni,nj -> nij', MPN,MPN)
    MPTT_nijr = 0.5*(np.einsum('ni,jr -> nijr',MPN,delta)
                    +np.einsum('nj,ir ->njir',MPN,delta)
                    -2*np.einsum('ni,nj,nr ->nijr',MPN,MPN,MPN))
    NPM=np.zeros_like(MPN)
    #attempt to create MPTM and MPTL instead ij sumbfor card coords
    #lengthm=0.0
    #while lengthm==0 :# DO WHILE (lengthm .lt. epsilon(lengthm)
    #calcdummy=MPN.shape
    #randomvec=np.ones((a_length,n_dim))#since its just for the 3d ?
    #MPTM_nijm=randomvec-np.einsum('niMPN,randomvec)*MPN
    #lengthm=np.sqrt(MPTM_nij,MPTM_nij)
    #MPTM_nij=MPTM_nijm/lengthm
    #MPTL_ij=np.cross(MPTM_nij,MPNN_nij)
    #MPTM= np.einsum('nij,)
    #MPTL=MPNN_nij-MPNV

In [4]:
#for the creation of m&l we need a 3dlinspace apriori with set e1 to get the projection ?
#normal strain vector for each microplane
if split==True :
    e_N = np.einsum('nij,...ij->...n',IntegScheme.MPNN_nij,epsilon_ij)
    MPTT_ijr =  IntegScheme.MPTT_nijr 
    e_V_norm = np.einsum('...ij,...ij->...',delta, epsilon_ij)/3
    e_V = np.einsum('ij,i->ij', np.ones_like(e_N), e_V_norm)
    e_D = e_N - e_V
    MPN = IntegScheme.MPN
    e_T_r=np.einsum('nija,...ij->...na',MPTT_ijr,epsilon_ij)
else :
    e_N = np.einsum('nij,...ij->...n',IntegScheme.MPNN_nij,epsilon_ij)
    MPTT_ijr =  IntegScheme.MPTT_nijr 
    e_T_r=np.einsum('nija,...ij->...na',MPTT_ijr,epsilon_ij)

In [5]:
%matplotlib widget
cm = 1 / 2.54  # centimeters in inches
fontlabel=20
labl=5
if split==True :
    plt.figure(figsize=(20*cm, 10*cm))
    rads = np.arange(0, (2 * np.pi), (2 * np.pi) / n_mp)
    plt.subplot(131, projection='polar')#subplot like matlab sets a matrix of plots but without setting ann array just the xyp where p position 
    #plt.subplot(131 , projection='polar')
    plt.plot(rads, e_V[-1, :], 'k')
    plt.ylim(-1.5 * np.max(np.abs(e_V)),
                    1.5 * np.max(np.abs(e_V)))
    plt.title(r'$ε_V$', fontsize=fontlabel)
    plt.tick_params(axis='both', which='major', labelsize=labl)

    plt.subplot(132, projection='polar')
    plt.plot(rads, e_D[-1, :], 'k')
    plt.ylim(-1.5 * np.max(np.abs(e_D)),
                    1.5 * np.max(np.abs(e_D)))
    plt.title(r'$\varepsilon_D$', fontsize=20)

    plt.tick_params(axis='both', which='major', labelsize=labl)
    plt.subplot(133, projection='polar')
    plt.plot(rads, np.linalg.norm(e_T_r[-1, :],axis=-1), 'k')
    plt.title(r'$ε_T$', fontsize=fontlabel)
    plt.tick_params(axis='both', which='major', labelsize=5)
else :
    plt.figure(figsize=(20*cm, 10*cm))
    rads = np.arange(0, (2 * np.pi), (2 * np.pi) / n_mp)
    plt.subplot(121, projection='polar')#subplot like matlab sets a matrix of plots but without setting ann array just the xyp where p position 
    plt.plot(rads, e_N[-1, :], 'k')
    plt.ylim(-1.5 * np.max(np.abs(e_N)),
                    1.5 * np.max(np.abs(e_N)))
    plt.title(r'$ε_N$', fontsize=fontlabel)
    plt.tick_params(axis='both', which='major', labelsize=labl)

    plt.subplot(122, projection='polar')
    plt.plot(rads, np.linalg.norm(e_T_r[-1, :],axis=-1), 'k')
    plt.title(r'$ε_T$', fontsize=fontlabel)
    plt.tick_params(axis='both', which='major', labelsize=labl)
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
#ELASTICITY CALCS
E_modulus =44800
n_poisson = 0.18
#issuewith if does not exit properly
if n_poisson >=-1 and n_poisson<0.25 : 
    K=E_modulus/(2*(1+n_poisson))
    G=E_modulus/(3*(1-2*n_poisson))
else:
    K=E_modulus/(2*(1+n_poisson))
    G=E_modulus/(3*(1-2*n_poisson))
    K_1=K
    K=E_modulus/(2*(1+n_poisson))
    G_1=0
    K_11 =K+K_1
    G_11 =G+G_1
    n_pseudopoisson=(3*K*(1+G/G_1)-2*G)/(6*K_*(1+G/G_1)+2*G)
    n_poisson=n_pseudopoisson
E_N = E_modulus/(1-2*n_poisson)
E_T = E_N*((1-4*n_poisson)/(1+n_poisson))
E_V= E_modulus/(1-2*n_poisson)
xo=1#for double constraint
E_D=5*E_modulus/((16+n_poisson)*(2+3*n_poisson))
# E_N = K/3

In [7]:
s_N = E_N * e_N
if split==True:
    s_V = E_V * e_V
    s_D = E_D * e_D
s_T_r = E_T * e_T_r

In [8]:
%matplotlib widget
cm = 1 / 2.54  # centimeters in inches
microplane = 45 #46°
if split==True :
    plt.figure(figsize=(20*cm, 15*cm))
    rads = np.arange(0, (2 * np.pi), (2 * np.pi) / n_mp)
    plt.subplot(131, projection='polar')
    plt.plot(rads, s_V[-1, :], 'k')
    plt.title(r'$\sigma_V$', fontsize=fontlabel)
    plt.tick_params(axis='both', which='major', labelsize=labl)

    plt.subplot(132, projection='polar')
    plt.plot(rads, s_D[-1, :], 'k')
    plt.title(r'$\sigma_D$', fontsize=fontlabel)
    plt.tick_params(axis='both', which='major', labelsize=labl)

    plt.subplot(133, projection='polar')
    plt.plot(rads, np.linalg.norm(s_T_r[-1, :],axis=-1), 'k')
    plt.title(r'$\sigma_T$', fontsize=fontlabel)
    plt.tick_params(axis='both', which='major', labelsize=labl)
else :
    plt.figure(figsize=(30*cm, 15*cm))
    rads = np.arange(0, (2 * np.pi), (2 * np.pi) / n_mp)
    plt.subplot(121, projection='polar')
    plt.plot(rads, s_N[-1, :], 'k')
    plt.title(r'$\sigma_N$', fontsize=fontlabel)
    plt.tick_params(axis='both', which='major', labelsize=labl)

    plt.subplot(122, projection='polar')
    plt.plot(rads, np.linalg.norm(s_T_r[-1, :],axis=-1), 'k')
    plt.title(r'$\sigma_T$', fontsize=fontlabel)
    plt.tick_params(axis='both', which='major', labelsize=labl)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
if split==True:
        sigma_ij= (     np.einsum('...n,...ij->...ij',s_V,delta)+np.einsum('...n,...n,...ni,nj->...ij',
                                  IntegScheme.MPW, s_D, IntegScheme.MPN, IntegScheme.MPN) +
                         0.5 * (np.einsum('n,...nf,na,fb->...ab',
                                           IntegScheme.MPW, s_T_r, IntegScheme.MPN, delta) +
                                 np.einsum('n,...nf,nb,fa->...ab',
                                           IntegScheme.MPW, s_T_r, IntegScheme.MPN, delta)
                         )
                )
else :        
        sigma_ij = (
                        np.einsum('n,...n,na,nb->...ab',
                                  IntegScheme.MPW, s_N, IntegScheme.MPN, IntegScheme.MPN) +
                        0.5 * (
                                np.einsum('n,...nf,na,fb->...ab',
                                          IntegScheme.MPW, s_T_r, IntegScheme.MPN, delta) +
                                np.einsum('n,...nf,nb,fa->...ab',
                                          IntegScheme.MPW, s_T_r, IntegScheme.MPN, delta)
                        )
                )

 

In [28]:

#Layout for the microplane calcs whith according to MS1 and Bazzant plasticity framework 
# to do
import numpy as np
import matplotlib.pyplot as plt 

################################################################
#DOMAIN GEN
#################################################################
zita=0 #faligue par
split=True
n_dim = 2 #number of dims in domain 
n_mp = 360 #number of microplanes in interpolation 
n_steps = 1000 #solver steps 
#ratio=step of defining actual domain
delta = np.identity(n_dim) #Kronecker δij
epsilon_11=np.linspace(0,0.01,n_steps)#define evenly the lin space from 0 to 0.01 with n_steps is the n in  nijk
#0.01 optional or otherwise raises DeprecationWarning when num cannot be safely interpreted as an integer
epsilon_ij=np.zeros((len(epsilon_11,),n_dim,n_dim))#creation of the finite 2d space
#question one-> since we use the ntr transormation and the space is also 2d there is no need for 3d?
#epsilon_ijk=np.zeros((len(epsilon_11,),n_dim,n_dim,n_dim))
epsilon_ij[:,0,0]=epsilon_11 #reshape,arrange,apprehend 
#epsilon_ij[:,0,0,0]=epsilon_11 for 3d
#incr=
#deij=epsilon_ij+incr
################################################################
#INTEGSCHEMES
################################################################
class IntegScheme():
    #n_mp=360
    alpha_list = np.linspace(0,2*np.pi,n_mp)#polar angle dom
    a_length=alpha_list.size 
    MPN = np.array([[np.cos(alpha),np.sin(alpha)] for alpha in alpha_list ])
    #alpha=float val?
    #weights of each microplane
    if n_dim==2:
        MPW = np.ones(n_mp) / n_mp * 2 
    elif n_dim==3:
        MPW = np.ones(n_mp) / n_mp * 6
#     else: 
    MPM = np.zeros_like(MPN)
    delta= np.identity(n_dim) #should be already set?
    MPNN_nij = np.einsum('ni,nj -> nij', MPN,MPN)
    MPTT_nijr = 0.5*(np.einsum('ni,jr -> nijr',MPN,delta)
                    +np.einsum('nj,ir ->njir',MPN,delta)
                    -2*np.einsum('ni,nj,nr ->nijr',MPN,MPN,MPN))
    NPM=np.zeros_like(MPN)
################################################################
#ELASTICITY CALCS
################################################################
E_modulus =44800
E_modulus =44800
n_poisson = 0.18
#issuewith if does not exit properly
if n_poisson >=-1 and n_poisson<0.25 : 
    K=E_modulus/(2*(1+n_poisson))
    G=E_modulus/(3*(1-2*n_poisson))
else:
    K=E_modulus/(2*(1+n_poisson))
    G=E_modulus/(3*(1-2*n_poisson))
    K_1=K
    K=E_modulus/(2*(1+n_poisson))
    G_1=0
    K_11 =K+K_1
    G_11 =G+G_1
    n_pseudopoisson=(3*K*(1+G/G_1)-2*G)/(6*K_1*(1+G/G_1)+2*G)
    n_poisson=n_pseudopoisson
E_N = E_modulus/(1-2*n_poisson)
E_T = E_N*((1-4*n_poisson)/(1+n_poisson))
E_V= E_modulus/(1-2*n_poisson)
xo=1#for double constraint
E_D=5*E_modulus/((16+n_poisson)*(2+3*n_poisson))

################################################################
#DISPS
################################################################
#for the creation of m&l we need a 3dlinspace apriori with set e1 to get the projection ?
#normal strain vector for each microplane
if split==True :
    e_N = np.einsum('nij,...ij->...n',IntegScheme.MPNN_nij,epsilon_ij)
    MPTT_ijr =  IntegScheme.MPTT_nijr 
    e_V_norm = np.einsum('...ij,...ij->...',delta, epsilon_ij)/3
    e_V = np.einsum('ij,i->ij', np.ones_like(e_N), e_V_norm)
    e_D = e_N - e_V
    MPN = IntegScheme.MPN
    e_T_r=np.einsum('nija,...ij->...na',MPTT_ijr,epsilon_ij)
else :
    e_N = np.einsum('nij,...ij->...n',IntegScheme.MPNN_nij,epsilon_ij)
    MPTT_ijr =  IntegScheme.MPTT_nijr 
    e_T_r=np.einsum('nija,...ij->...na',MPTT_ijr,epsilon_ij)
# print((epsilon_ij[1:] - epsilon_ij[:-1]).shape)
# de_ij= np.eye(epsilon_11,n_dim)#for sure wrong just to continue with the script 
de_ij= ((epsilon_ij[1:] - epsilon_ij[:-1]))
de_v=np.einsum('...ij,...ij->...',de_ij, de_ij)/n_dim
# print(IntegScheme.MPNN_nij.shape)
de_n=np.einsum('...nij,...ij->...n',IntegScheme.MPNN_nij,de_ij)
de_tr=np.einsum('...nija,...ij->...na',MPTT_ijr,de_ij)

################################################################
#ELASTIC PRED
################################################################
s_N = E_N * e_N
if split==True:
    s_V = E_V * e_V
    s_D = E_D * e_D
s_T_r = E_T * e_T_r

################################################################
#PARAMETERS AND INITIALASATION
################################################################
k_1=150*10**-6
k_2=110000
k_3=30
k_4=100
k_5=10**-4
f_c0dot=20  #MPa
f_cdot=20   #Mpa 
fc0= 15.08         #MPa Reference compressive strength
c_1= 8.9*10**-2    # Controls the uniaxial tensile strength
c_2= 17.6*10**-2   # Controls the roundness of the peak in uniaxial tension
c_3= 4             # Controls the slope of the postpeak in uniaxial tension
c_4= 50            # Controls the slope of the postpeak tail in uniaxial compression
c_5= 3500          # Controls the volumetric expansion under compression
c_6= 20            # Controls the roundness of the peak in volumetric expansion under compression
c_7= 1             # Controls the slope of the initial postpeak in uniaxial compression
c_8= 8             # Controls the peak strength in uniaxial compression
c_9= 1.2*10**-2    # Controls the peak roundness in uniaxial compression
c_10= 0.33         # Controls the effective friction coefficient
c_11= 0.5          # Initial cohesion in frictional response
c_12= 2.36         # Controls the change of cohesion with tensile volumetric strains
c_13= 4500         # Controls the unloading slope in tension
c_14= 300          # Controls the unloading slope at low hydrostatic compression
c_15= 4000         # Controls the transition from unloading slope at high confinement to that at low confinement
c_16= 60           # Controls the unloading slope at high hydrostatic compression
c_17= 1.4          # Controls the tensile strength
c_18= 1.6*10**-3   # Controls the tensile cracking under compression
c_19= 1000         # Controls the tensile softening rate induced by compression
c_20= 1.8          # Controls the volumetric-deviatoric coupling at high pressures
c_21=250
E_D0=E_D
gama_0=f_c0dot/E_D0+f_cdot/E_D
E_N0 = E_N
e_0Nplus=np.zeros(n_mp)
e_0Nminus=np.zeros(n_mp)
s_D_boundtrack=np.zeros((n_steps,n_mp))
s_N_boundtrack=np.zeros((n_steps,n_mp))
s_V_boundtrack=np.zeros((n_steps,n_mp))
s_T_boundtrack=np.zeros((n_steps,n_mp))
#loop over each time step
for j in range(n_steps): 
    #loop over each microplane
    for i in range(n_mp):
        epsI = np.max(np.linalg.eig(epsilon_ij[i])[0])
        #epsI = np.linalg.eig(epsilon_ij[j-1])[0,0]
        #epsII = np.linalg.eig(epsilon_ij[j-1])[1,0]
        epsII = np.min(np.linalg.eig(epsilon_ij[i])[0])
        if j>0:
            
             e_elastic=-s_V[-1,i]/E_N[-1,i]
    #NOTE: this is not correct! Just by simply printing the value of epsI you should realize that it is not the first eigenvalue since you get a multidimensional array
    # Eingenvalue is a scalar, so it should be a float number. I have just checked the documentation of linalg.eig and corrected the implementation
    #//==>   epsII = np.min(np.linalg.eig(epsilon_ij[i])[0])
   


        ##########################################################
        #compressive deviatoric boundary Eq. 15 - 18
        ##########################################################
        # predictor for compressive deviatoric stress
        #s_D[j][i] = E_D * e_D[j][i]
        s_D[j][i] = E_modulus * e_D[j][i]
        # obtaining deviatoric compressive strain boundary (threshold)
        gama_1=np.exp(gama_0)*np.tanh(c_9*(-e_V[j][i])/k_1)
        bita_2=c_5*gama_1+c_7
        bita_3=c_6*gama_1+c_8
        s_D_bound=E_D*k_1*bita_3/(1+6*((-e_D[j][i])/k_1*bita_2)**2)
        s_D_boundtrack[j][i]=s_D_bound
        # checking if we are crossing the boundary for time step j and microplane i
        if e_N[j][i] < 0:
            # print("comp.")
            # if we cross boundary, correct microplane i stress
            if s_D[j][i] < s_D_bound:
                # corrector deviatoric stress
                s_D[j][i] = s_D_bound

        ##########################################################
        #volumentric boundary Eq. 12 - 14
        ##########################################################
        de_V = e_V[j][i] - e_V[j-1][i] # delta volumetric strain
        e_elastic = np.max([-s_V[j-1][i]/E_N0, 0])
        a=(k_5/(1+np.min([-s_V[j][i],c_21])/E_N))*((epsI-epsII)/k_1)**(c_20)+k_4
        # print(k_5, -s_V[i][j],c_21,E_N, (epsI-epsII)/k_1, c_20, k_4)
        s_V_bound=-E_modulus*k_1*k_3*np.exp(-e_V[j][i]/(k_1*a))
        s_V_boundtrack[j][i]=s_V_bound
        s_V[j][i] = s_V[j-1][i] + E_modulus * de_V
        # print(s_V_bound, 'bound')
        # print(s_V[i][j], 'stress')
        if s_V[j][i] > s_V_bound:
            s_V[j][i] = s_V_bound
        ##########################################################
        #tensile boundary Eq. 19 - 24
        ##########################################################
        de_N = e_N[j][i] - e_N[j-1][i]
        if s_N[j-1][i]>=0:      #np.zero_like(s_N)
            if de_N > 0:
                E_N=E_N0*np.exp(-c_13*e_0Nplus)*(1-0.1*zita**2)**-1
            else:
                E_N = E_N0
        else:
            E_N=E_N0*(np.exp(-c_14*np.abs(e_0Nminus)/(1+c_15*e_elastic))+c_16*e_elastic)
            
        s_N[j][i] = s_N[j-1][i] + E_N * de_N
        
        if s_N[j][i] > 0:
            bita_1=-c_1+c_17*np.exp(-c_19*(-s_V[j-1][i]-c_18)/E_N0)
            s_N_bound=E_modulus*k_1*bita_1*np.exp(-(e_N[j][i]-bita_1*c_2*k_1)/(-c_4*e_elastic+k_1*c_3))
            s_N_boundtrack[j][i]=s_N_bound
            if s_N[j][i] > s_N_bound: 
                s_N[j][i]=np.max([np.min([s_N[j][i],s_N_bound]),s_V_bound+s_D_bound])
                e_0Nminus = np.amin(e_N[i])
                e_0Nplus = np.amax(e_N[i])
        ##########################################################
        #Shear boundary Eq.  24-32
        ##########################################################

        s_T_r_0=s_T_r[j-1][i]      

        E_T = E_N*((1-4*n_poisson)/(1+n_poisson))         
        s_N0dot=E_T*k_1*c_11-c_12*e_V#shouldnt it be inside the microplane loop 
        if s_N[j][i]<=0:
            s_T_bound=(c_10*(s_N0dot-s_N)**(-1)+(E_T*k_1-k_2)**(-1))**(-1)
        else:
            s_T_r_bound=(c_10*s_N0dot**(-1)+(E_T*k_1-k_2)**(-1))**(-1)
            s_T_r_elastic=((s_T_r +E_T*de_tr)**2)**-1
            s_T_r=np.minimum(s_T_r_bound,np.abs(s_T_r_elastic))
    s_V[j,:]=np.einsum('...i,i',s_N,IntegScheme.MPW)

# if split==True:
#         sigma_ij= (     np.einsum('i,ij',s_V,delta)+np.einsum('d,...d,na,nb->...ab',
#                                   IntegScheme.MPW, s_D, IntegScheme.MPN, IntegScheme.MPN) +
#                         0.5 * (
#                                 np.einsum('n,...nf,na,fb->...ab',
#                                           IntegScheme.MPW, s_T_r, IntegScheme.MPN, delta) +
#                                 np.einsum('n,...nf,nb,fa->...ab',
#                                           IntegScheme.MPW, s_T_r, IntegScheme.MPN, delta)
#                         )
#                 )
# else :        
#         sigma_ij = (
#                         np.einsum('n,...n,na,nb->...ab',
#                                   IntegScheme.MPW, s_N, IntegScheme.MPN, IntegScheme.MPN) +
#                         0.5 * (
#                                 np.einsum('n,...nf,na,fb->...ab',
#                                           IntegScheme.MPW, s_T_r, IntegScheme.MPN, delta) +
#                                 np.einsum('n,...nf,nb,fa->...ab',
#                                           IntegScheme.MPW, s_T_r, IntegScheme.MPN, delta)
#                         )
#                 )

            






ValueError: could not broadcast input array from shape (1000) into shape (360)